In [124]:
# we need the following libraries, so let's install them
%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow
%pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [125]:
# importing libraries
from time import time
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import skimage
from  imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import sys
import warnings
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
import logging
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# from google.colab import drive
# drive.mount('/content/gdrive')

In [126]:
# loadnig the data and removing the useless column 'Unnamed: 0'
# whole_data = pd.read_csv('/content/gdrive/MyDrive/radiomics/Chexpert_synced_data_nan_free.csv')
whole_data = pd.read_csv('Chexpert_synced_data_nan_free.csv')
whole_data.head(5)

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,-669.603949,685.781964,1.059520e+10,6.017600,866.092155,1.713989,934.117676,428.763596,10.279660,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,-801.474548,564.486328,1.274714e+10,6.146630,954.811813,1.756523,1020.768616,468.058052,-132.958831,...,0.0,1.0,1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0
2,2,-844.417249,307.704349,9.364581e+09,5.848908,690.788330,1.913630,589.552979,354.572276,-308.642302,...,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
3,4,-742.036096,673.796082,1.132259e+10,6.076017,1038.149048,1.627926,933.427429,467.972161,-17.909900,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,5,-802.322357,560.980408,1.410547e+10,6.061123,783.288322,2.001557,869.513733,412.838385,-210.721353,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0


In [128]:
# excludiung Ages with Nans
temporary_labels = np.zeros((whole_data.shape[0],))
for i, val in enumerate(whole_data['Race'].values):
    if 'White' in val:
        temporary_labels[i] = 0
    elif 'Black' in val or 'African American' in val:
        temporary_labels[i] = 1
    elif 'Asian' in val:
        temporary_labels[i] = 2
    else:
        temporary_labels[i] = np.nan


In [129]:
# the number of each label:
# white, african american, asian
(temporary_labels==0).sum(), (temporary_labels==1).sum(), (temporary_labels==2).sum()

(106089, 9921, 19396)

In [130]:
whole_data['new_race'] = temporary_labels.tolist()

In [132]:
whole_data = whole_data[~pd.isnull(whole_data['new_race'])]

In [134]:
random_seed = 21202
del whole_data['Unnamed: 0']
whole_data.head()
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)

In [135]:
# finding total number of samples:
n_samples = whole_data.shape[0]
print("number of samples are: {}".format(n_samples))

number of samples are: 135406


In [136]:
# names of all targets:
targets_names = list(whole_data.iloc[:,464:])
print(targets_names)

['index', 'Path', 'Patient_id', 'Study_Number', 'Sex', 'Age', 'Race', 'Ethnicity', 'Frontal/Lateral', 'AP/PA', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices', 'new_race']


In [90]:
# finding the number of unique subjects:
unique_subjects = pd.unique(whole_data['Patient_id'])
print('The number of unique subjects are: {0}'.format(len(unique_subjects)))

The number of unique subjects are: 46107


In [137]:
# preallocating the frequency of session of the subjects:
# this is written to make sure that we dont use a subject in both training and testing split.
test_split = 0.06       # precent out of total samples
valid_split = 0.1       # precent out of total samples (this is the sumation of both training and validation splits)

n_test = n_samples * test_split
n_whole_train = n_samples - n_test

frequencies = np.ones((unique_subjects.shape[0], 4)) * np.nan
for i, sbj in tqdm(enumerate(unique_subjects)):
  frequencies[i, 0] = sbj
  frequencies[i, 1] = (whole_data['Patient_id'] == sbj).sum()
cumolative_sums_of_samples = np.cumsum(frequencies[:,1])
frequencies[:, 2] = cumolative_sums_of_samples
frequencies[:, 3] = cumolative_sums_of_samples/cumolative_sums_of_samples[~0]

46107it [00:11, 3871.91it/s]


In [138]:
# visualizing the frequency  of subjects and images taken from each
vis_frequencies = pd.DataFrame(frequencies,columns=['subject_ID','n_of_sessions/images','cumolative_sessions','preccent_of_number_of_images'])
vis_frequencies.head(15)

,subject_ID,n_of_sessions/images,cumolative_sessions,preccent_of_number_of_images
0,38202.0,3.0,3.0,0.000022
1,10143.0,3.0,6.0,0.000044
2,28750.0,1.0,7.0,0.000052
3,56970.0,2.0,9.0,0.000066
4,37884.0,23.0,32.0,0.000236
5,33968.0,3.0,35.0,0.000258
6,7770.0,5.0,40.0,0.000295
7,5640.0,4.0,44.0,0.000325
8,11368.0,2.0,46.0,0.000340
9,46543.0,2.0,48.0,0.000354


In [139]:
# separating the testing split without having any mutual_subjects:
testing_subjects_masking = vis_frequencies['preccent_of_number_of_images'] < test_split
testing_subjects = vis_frequencies[testing_subjects_masking]['subject_ID'].values.tolist()
whole_train_subjects = vis_frequencies[~testing_subjects_masking]['subject_ID'].values.tolist()

In [140]:
# excluding irrelevant features
excluding2ds =  np.concatenate( (np.arange(32,41,dtype=int),
                                 np.arange(32,41,dtype=int)+ 116,
                                 np.arange(32,41,dtype=int)+116*2,
                                 np.arange(32,41,dtype=int)+116*3))

whole_data.drop(whole_data.columns[excluding2ds.tolist()],axis = 1,inplace=True)

In [141]:
# separating test_data and whole_trainig_data
whole_data = whole_data.sample(frac=1.0, random_state=random_seed)
Test = whole_data[whole_data['Patient_id'].isin(testing_subjects)]
X_test, y_test = Test.iloc[:,:464-36].values, Test['new_race'].values

# separating test_data and whole_trainig_data
Train = whole_data[whole_data['Patient_id'].isin(whole_train_subjects)]

# the size of training/testing_split:
print('The size of the testing split is: {}'.format(Test.shape))
print('The size of the whole trainig (train + valid) split is: {}'.format(Train.shape))

The size of the testing split is: (8081, 453)
The size of the whole trainig (train + valid) split is: (127325, 453)


In [142]:
# splitting the validation split:
X_train, X_val, y_train, y_val = train_test_split(Train.iloc[:,:464-36].values,
                                                  Train['new_race'].values,
                                                  test_size=valid_split,
                                                  random_state=random_seed
                                                  )

In [143]:
# checking the number of nans:
pd.isnull(X_train).sum(), pd.isnull(X_test).sum(), pd.isnull(X_val).sum()

(0, 0, 0)

In [144]:
# checking the number of classes in each split:
# train
(y_train==0).sum(),(y_train==1).sum(),(y_train==2).sum(), (y_val==0).sum(),(y_val==1).sum(),(y_val==2).sum()

(89488, 8374, 16730, 9939, 977, 1817)

In [145]:
(y_test==0).sum(),(y_test==1).sum(),(y_test==2).sum()

(6662, 570, 849)

In [146]:
print('The size of the validation split is: {}'.format(X_val.shape))
X_train.shape, X_val.shape, y_train.shape, y_val.shape

The size of the validation split is: (12733, 428)


((114592, 428), (12733, 428), (114592,), (12733,))

In [147]:
sm = SMOTE(random_state=21202)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [157]:
y_train  = pd.get_dummies(pd.DataFrame(y_train.tolist(),columns=['column_name']),columns=['column_name']).values * 1
y_val  = pd.get_dummies(pd.DataFrame(y_val.tolist(),columns=['column_name']),columns=['column_name']).values * 1
y_test  = pd.get_dummies(pd.DataFrame(y_test.tolist(),columns=['column_name']),columns=['column_name']).values * 1

In [158]:
# the number of samples after resampling:
X_train.shape, y_train.shape

((268464, 428), (268464, 3))

In [169]:
def classifer_1(features_number: int) -> callable:
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=100, activation="relu", input_dim=features_number))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=300, activation="relu"))
    model.add(keras.layers.Dropout(0.0))
    model.add(keras.layers.Dense(units=3,activation='softmax'))
    model.summary()
    return model

def compiler_1(X_train_, Y_train_, X_valid_, Y_valid_,mdl_callback):
  initial_learning_rate = 0.0001
  epochs = 500
  batch_size = 128
  model = classifer_1(features_number=X_train_.shape[1])
  # lss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
  lss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
  metrics = ['accuracy',
                        tf.keras.metrics.AUC(curve='PR',name='PR-curve'),
                        tf.keras.metrics.AUC(curve='ROC',name='ROC-curve')]
  # metrics = metrics=[tf.keras.metrics.RootMeanSquaredError()]
  # metrics = [tf.keras.metrics.RootMeanSquaredError()]
  # lss = 'mean_absolute_error'
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-08),
                                                loss=lss,
                metrics=metrics)
  model.fit(x=X_train_,
            y=Y_train_,
            epochs=epochs,
            initial_epoch=0,
            batch_size=batch_size,
            validation_data=(X_valid_, Y_valid_),
            verbose=2,
            shuffle=True,
            callbacks=[mdl_callback])
  return model

In [170]:
# full Model

X_train_ = X_train.copy()
X_val_ = X_val.copy()
X_test_ = X_test.copy()

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + 'full'+ '_'
saving_path = 'chexpert_race_3categ_' + 'full'+ '_'

belongings={'mean':mean_val,
            'feature_number':None,
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 100)               42100     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_16 (Dense)            (None, 300)               30300     
                                                                 
 dropout_11 (Dropout)        (None, 300)               0         
                                                                 
 dense_17 (Dense)            (None, 3)                 903       
                                                                 
Total params: 73303 (286.34 KB)
Trainable params: 73303 (286.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [171]:
# each segments:

segment_number = 0

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}
np.save(saving_path + 'belongings.npy',belongings)
checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 100)               10600     
                                                                 
 dropout_12 (Dropout)        (None, 100)               0         
                                                                 
 dense_19 (Dense)            (None, 300)               30300     
                                                                 
 dropout_13 (Dropout)        (None, 300)               0         
                                                                 
 dense_20 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [172]:
# each segments:
segment_number = 1

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'
# checkpoint_filepath = '_mdl.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               10600     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 300)               30300     
                                                                 
 dropout_15 (Dropout)        (None, 300)               0         
                                                                 
 dense_23 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [173]:
# each segments:
segment_number = 2

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'
saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               10600     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_25 (Dense)            (None, 300)               30300     
                                                                 
 dropout_17 (Dropout)        (None, 300)               0         
                                                                 
 dense_26 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [174]:
# each segments:
segment_number = 3

segment_names = ['segment_0_','segment_1_','segment_2_','segment_3_']

X_train_ = X_train.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_val_ = X_val.copy()[:, 107*segment_number:107*(segment_number + 1)]
X_test_ = X_test.copy()[:, 107*segment_number:107*(segment_number + 1)]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

# saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_segment_' + segment_names[segment_number]+ '_'

saving_path = 'chexpert_race_3categ_segment_' + segment_names[segment_number]+ '_'


belongings={'mean':mean_val,
            'feature_number':segment_names[segment_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_0_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 100)               10600     
                                                                 
 dropout_18 (Dropout)        (None, 100)               0         
                                                                 
 dense_28 (Dense)            (None, 300)               30300     
                                                                 
 dropout_19 (Dropout)        (None, 300)               0         
                                                                 
 dense_29 (Dense)            (None, 3)                 903       
                                                                 
Total params: 41803 (163.29 KB)
Trainable params: 41803 (163.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500

Epo

KeyboardInterrupt: 

In [ ]:
fos_features = (np.arange(0,18) + 0).tolist() + (np.arange(0,18) + 107).tolist() + (np.arange(0,18) + 107*2).tolist() + (np.arange(0,18) + 107*3).tolist()
_3d_features = (np.arange(18,32) + 0).tolist() + (np.arange(18,32) + 107).tolist() + (np.arange(18,32) + 107*2).tolist() + (np.arange(18,32) + 107*3).tolist()
glcm_features = (np.arange(32,56) + 0).tolist() + (np.arange(32,56) + 107).tolist() + (np.arange(32,56) + 107*2).tolist() + (np.arange(32,56) + 107*3).tolist()
glszm_features = (np.arange(56,72) + 0).tolist() + (np.arange(56,72)+ 107).tolist() + (np.arange(56,72) + 107*2).tolist() + (np.arange(56,72) + 107*3).tolist()
glrlm_features = (np.arange(72,88) + 0).tolist() + (np.arange(72,88)  + 107).tolist() + (np.arange(72,88)  + 107*2).tolist() + (np.arange(72,88)  + 107*3).tolist()
ngtdm_features = (np.arange(88,93) + 0).tolist() + (np.arange(88,93)  + 107).tolist() + (np.arange(88,93)  + 107*2).tolist() + (np.arange(88,93) + 107*3).tolist()
gldm_features = (np.arange(93,107) + 0).tolist() + (np.arange(93,107)  + 107).tolist() + (np.arange(93,107)  + 107*2).tolist() + (np.arange(93,107) + 107*3).tolist()
feature_names = ['fos_features', '_3d_features', 'glcm_features', 'glszm_features', 'glrlm_features', 'ngtdm_features', 'gldm_features']
feature_indexes = [fos_features, _3d_features, glcm_features, glszm_features, glrlm_features, ngtdm_features, gldm_features]

In [ ]:
feature_number = 0

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_0 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 1

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_1 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 2

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_2 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 3

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_3 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 4

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_4 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 5

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_5 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)

In [ ]:
feature_number = 6

X_train_ = X_train.copy()[:,feature_indexes[feature_number]]
X_val_ = X_val.copy()[:,feature_indexes[feature_number]]
X_test_ = X_test.copy()[:,feature_indexes[feature_number]]

mean_val = X_train_.mean(axis=0)
std_val = X_train_.std(axis=0)

# excluding invariant variables:
mask_excluded = ~(std_val==0)
X_train_[:,mask_excluded] -= mean_val[mask_excluded]
X_train_[:,mask_excluded] /= std_val[mask_excluded]

X_val_[:,mask_excluded] -= mean_val[mask_excluded]
X_val_[:,mask_excluded] /= std_val[mask_excluded]

X_test_[:,mask_excluded] -= mean_val[mask_excluded]
X_test_[:,mask_excluded] /= std_val[mask_excluded]

# excluding invariant variables:
mask_excluded = ~(std_val==0)

saving_path = '/content/gdrive/MyDrive/radiomics/age_chexpert/chexpert_sex_' + feature_names[feature_number]+ '_'

belongings={'mean':mean_val,
            'feature_number':feature_names[feature_number],
            'std':std_val,
            'mask_excluded':mask_excluded,
            'x_test':X_test_,
            'y_test':y_test}

np.save(saving_path + 'belongings.npy',belongings)

checkpoint_filepath = saving_path +'_mdl.h5'

model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
                                                                filepath=checkpoint_filepath,
                                                                save_weights_only=False,
                                                                monitor='val_loss',
                                                                save_best_only=True,
                                                                save_freq='epoch',
                                                                verbose=2
                                                            )

model_1_6 = compiler_1(X_train_ = X_train_[:,mask_excluded],
                    Y_train_ = y_train,
                    X_valid_ = X_val_[:,mask_excluded],
                    Y_valid_ = y_val,
                    mdl_callback=model_checkpoint_callback_1)